# <center>Bing Image Downloader</center>

Import the required modules and functions

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os
import re
import csv
from datetime import datetime
from urllib.request import urlretrieve

In [2]:
# To create an empty csv file if it does not exists already, uncomment the below code
header = ['ImageID','Title','Date']

#with open('BingImages.csv','w',newline='', encoding='UTF8') as f:
#    writer = csv.writer(f)
#    writer.writerow(header)


#with open('Failed.csv','w',newline='', encoding='UTF8') as f:
#    writer = csv.writer(f)
#    writer.writerow(['ImageID'])


Download the homepage and find the number of pages in the website

In [3]:
homepage_url = 'https://windows10spotlight.com/'

homepage_url_data = requests.get(homepage_url).text

homepage_soup = BeautifulSoup(homepage_url_data,"html.parser")

number_of_pages = int(re.findall('\d+.\d+', homepage_soup.find_all("nav", {"class":"navigation pagination"})[0].text)[1])

home_directory = os.getcwd()

bing_df = pd.read_csv('BingImages.csv')

#countries_df = pd.read_csv('countries.csv')

Download each page in the website to collect the image id's in each of it

Clean the image caption

In [4]:
def clean_caption(caption):
    return caption.replace('/',' ').replace('\\',' ').replace('?','').replace('"','').replace("'",'')


In [5]:
#write the image_stamp to the csv
def report_success(id, title, img_date):

    data = [id, title, img_date]
    with open('BingImages.csv','a+',newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)


In [6]:

# to maintain a record of the failed downloads  
def report_failed(id):

    os.chdir(home_directory)
    data = [id]
    with open('Failed.csv','a+',newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)


In [7]:

def download_image_pc(link, caption, extension):

    os.chdir(r'.\Bing Images\Desktop Background')
    file_name = caption + '.' + extension
    urlretrieve(link, file_name)

    #link_data = requests.get(link)
    #path=os.path.join(os.getcwd() , file_name)
   
    #with open(path, 'wb') as f:
    #    print(link)
    #    print(file_name)
    #    f.write(link_data.content)

    os.chdir(home_directory)


#from urllib.request import urlretrieve

#url = 'https://windows10spotlight.com/wp-content/uploads/2022/08/fee98416ec0b1c608573ff159ee7a235.jpg'
#urlretrieve(url, 'image.jpg')


In [8]:

def download_image_mobile(link, caption, extension):

    os.chdir(r'.\Bing Images\Mobile Background')

    file_name = caption + '.' + extension
    urlretrieve(link, file_name)

    #link_data = requests.get(link)
    #path=os.path.join(os.getcwd() , file_name)

    #with open(path, 'wb') as f:
    #    print(link)
    #    f.write(link_data.content)
    
    os.chdir(home_directory)
    


In [9]:
def redirect_to_image(image_id):

    try:
        image_page_url = 'https://windows10spotlight.com/images/' + str(image_id)

        image_page_url_data = requests.get(image_page_url).text
        
        image_page_soup = BeautifulSoup(image_page_url_data, 'html.parser')

        #extract the date of the image
        dateStr = image_page_soup.find_all("span", {"class":"date"})[0].text.replace('-',' ')
        image_date = datetime.strptime(dateStr, '%d %b %Y').date()

        #extract the title of the image
        image_caption = clean_caption(image_page_soup.find('h1').text)
        #country = image_caption.split(',')[-1] 

        #extract the download links
        image_soup = image_page_soup.find("div", {"class":"entry"}).find_all('a')

        pc_image_link = image_soup[0].get('href')
        pc_image_extension = pc_image_link.split('.')[-1]

        mobile_image_link = image_soup[1].get('href')
        mobile_image_extension = mobile_image_link.split('.')[-1]


        download_image_pc(pc_image_link, image_caption, pc_image_extension)
        download_image_mobile(mobile_image_link, image_caption, mobile_image_extension)

        #save to the dataframe
        bing_df.loc[len(bing_df.index)] = [image_id, image_caption, image_date]

        print(image_id)
        
        report_success(image_id, image_caption, image_date)

    except:
        print('Error saving: {0}'.format(image_id))
        report_failed(image_id)
        


In [10]:

def redirect_to_page(page_number):

    page_url = 'https://windows10spotlight.com/page/' + str(page_number)

    page_url_data = requests.get(page_url).text

    page_soup = BeautifulSoup(page_url_data, 'html.parser')

    page_content_soup = page_soup.find(id='content')
    
    for link in page_content_soup.find_all('h2'):
        if(link.parent.name == 'article'):
            img_id = link.text

            if(img_id not in bing_df['ImageID'].values):
                redirect_to_image(img_id)


In [11]:

#browse through every page
for page in range(260,number_of_pages+1):
    
    print(page)
    redirect_to_page(page) #go to the specific page number

260
261
262
263
264
265
266
267
268
269
Error saving: 3663f6ac60804449c940323ea209af13
2534ceed85ab8ae47a3fca745372d403
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
